In [1]:
# Extract pcb names
import os

PCB_folder = "../../new_PCBs/"
pcb_names = []
for pcb_name in os.listdir(PCB_folder):
    pcb_names.append(pcb_name)
    print(pcb_name)

breakout-boards_swd-and-uart
Raspberry-Pi-Soft-Power-Controller_RetroPie Power Board v1a
xwhatits-capsense-controller_amkey-usb
eeg_brainboard_wearable_v2
eeg_brainboard_wearable
teensy-touch_teensy-touch
eurorack_headphones_eurorack_headphones
keyboard_converter_converter
eeg_brainboardv0
rfidBoard_rfid
eeg_muscleboardv0
breakout-boards_swd-to-wires
voltastream0-kicad_VoltaStreamZERO
Raspberry-Pi-Soft-Power-Controller_Zero Current Soft Power
NightClock_ESPBoard
nan-15_pcb
tepmachcha_tepmachcha
Raspberry-Pi-Soft-Power-Controller_Switching Supply TPS563208 MCI
xwhatits-capsense-controller_displaywriter-usb
xwhatits-capsense-controller_model-f-usb
xwhatits-capsense-controller_beamspring-usb
HACK_hack
xwhatits-capsense-controller_solenoid-driver
SAMPad_SAMPad
TCKB_Kicad_TCKB
breakout-boards_usb-5v-3v3
eeg_brainboardv1
nikon_gps_nikon_gps
breakout-boards_esp8266-jtag
eeg_brainboardv2
rfidBoard_POE
breakout-boards_avr-isp-x2
nan-15_top-plate
xwhatits-capsense-controller_model-f-3178-adaptor

In [4]:
# clean new_PCB folder
import os
import shutil

new_PCB_folder = "../../new_PCBs/"
pcb_names = list(os.listdir(new_PCB_folder))

for pcb_name in pcb_names:
    processed_name = os.path.join(new_PCB_folder, pcb_name, "processed.kicad_pcb")
    if not os.path.exists(processed_name):
        print(pcb_name)
        shutil.rmtree(os.path.join(new_PCB_folder, pcb_name))
    else:
        for file in os.listdir(os.path.join(new_PCB_folder, pcb_name)):
            if ".kicad_pcb" not in file and ".json" not in file:
                # print(os.path.join(new_PCB_folder, pcb_name, file))
                os.remove(os.path.join(new_PCB_folder, pcb_name, file))

# Read CSV

In [1]:
import csv
from functools import reduce

def read_csv(file_name):
    with open(file_name, "r") as f:
        pcb_names = list(csv.reader(f, delimiter=","))

    pcb_names = reduce(lambda xs, ys: xs + ys, pcb_names)
    return pcb_names

# Deal with fp_text

In [2]:
from collections import defaultdict

def revise_module_fp_text(pcb):

    text_count = defaultdict(int)
    for m in pcb.module:
        text_count[m.fp_text[0][1]] += 1

    print(text_count)
    for m in pcb.module:
        if text_count[m.fp_text[0][1]] > 1 or m.fp_text[0][1] == '""' or m.fp_text[0][1] == '" "':
            tmp_text = m.fp_text[0][1]
            if tmp_text == '""':
                m.fp_text[0][1] =  f"Noname_{text_count[tmp_text]}"
            elif tmp_text == '" "':
                m.fp_text[0][1] =  f"None_{text_count[tmp_text]}"
            else:
                m.fp_text[0][1] =  str(tmp_text) + "_" + str(text_count[tmp_text])
            del m.fp_text[0][1][0]
            text_count[tmp_text] -= 1

# Deal with fill zones

In [3]:
import sys
sys.path.append('..')
from Data_extraction.thirdparty.kicad_parser.kicad_pcb import *

import os

def delete_segment_via(pcb, delete_nets):

    seg_idx = []
    via_idx = []
    for i in range(len(pcb.segment)):
        if pcb.segment[i].net in delete_nets:
            seg_idx.append(i)
    seg_idx.sort()
    for i in range(len(pcb.via)):
        if pcb.via[i].net in delete_nets:
            via_idx.append(i)
    via_idx.sort()
    # print(seg_idx, via_idx)

    bc = 0
    for i in seg_idx:
        del_i = i - bc
        if pcb.segment[del_i].net in delete_nets:
            del pcb.segment[del_i]
            bc += 1
        else:
            print(f"{i} is not in the delete net for segment")

    bc = 0
    for i in via_idx:
        del_i = i - bc
        if pcb.via[del_i].net in delete_nets:
            del pcb.via[del_i]
            bc += 1
        else:
            print(f"{i} is not in the delete net for via")


pcb_names = read_csv("good_raw.csv")

print(len(pcb_names), pcb_names)

fill_zone_folder = "good_raws"
if not os.path.exists(fill_zone_folder):
    os.mkdir(fill_zone_folder)

for name in pcb_names:
    pcb_file_path = "../../PCBs_new/" + name + "/raw.kicad_pcb"
    new_pcb_name = name + ".kicad_pcb"
    pcb = KicadPCB.load(pcb_file_path)
    fill_zone_nets = set([])
    for zone in pcb.zone:
        fill_zone_nets.add(zone.net)
    print(name)
    del pcb.zone
    delete_segment_via(pcb, fill_zone_nets)
    revise_module_fp_text(pcb)
    pcb.export(os.path.join(fill_zone_folder, new_pcb_name))



63 ['kitspace_antenna_analyser', 'bobc_control_panel', 'bobc_matrix_clock', 'bobc_LCD-panel-adapter-lvc', 'bobc_mbeduinopresso', 'bobc_MS-F100', 'cnlohr_wiflier', 'kitspace_bbq10breakout', 'kitspace_BQ25570_Harvester', 'kitspace_Brk40p', 'kitspace_CA3306E', 'kitspace_CH330', 'kitspace_ChaosLooper', 'kitspace_CO2', 'kitspace_CocoMixtape_UGM_Kicad', 'kitspace_cseduinov4', 'kitspace_DIY_detector', 'kitspace_dynamixel_shield', 'kitspace_esp8266', 'kitspace_esp8266-12-breakout', 'kitspace_ESP8266-PowerMonitor', 'kitspace_f-91w', 'kitspace_fast_diode_pcb', 'kitspace_filaSens', 'kitspace_gas_sensor', 'kitspace_GPSMux', 'kitspace_grove_adaptor', 'kitspace_hbridge_driver', 'kitspace_Heltec_Node', 'kitspace_hum_temp_sensor', 'kitspace_ir_sensor', 'kitspace_Mixtape_Barebone', 'kitspace_nunchuk_breakout', 'kitspace_OutlineChaos', 'kitspace_PCBs', 'kitspace_pmt_combiner', 'kitspace_postcard', 'kitspace_Potentiometer_mount_4LED', 'kitspace_Potentiometer_mount_8LED', 'kitspace_Potentiometer_mount_16L

('expects list of more than 2 element', [988, 'layers'])
('expects list of more than 2 element', [1002, 'layers'])


kitspace_cseduinov4
defaultdict(<class 'int'>, {'IC1': 1, 'C4': 1, 'C3': 1, 'C5': 1, 'D1': 1, 'SW1': 1, 'X1': 1, 'REF**': 4, 'P5': 1, 'P4': 1, 'P1': 1, 'P3': 1, 'P2': 1, 'C1': 1, 'C2': 1, 'D2': 1, 'R1': 1, 'R2': 1, 'U1': 1})
kitspace_DIY_detector
defaultdict(<class 'int'>, {'J2': 1, 'J1': 1, 'REF***': 2, 'R8': 1, 'R9': 1, 'C9': 1, 'R3': 1, 'U1': 1, 'C8': 1, 'C1': 1, 'C2': 1, 'C3': 1, 'C4': 1, 'C5': 1, 'C6': 1, 'C7': 1, 'C10': 1, 'R2': 1, 'R4': 1, 'R5': 1, 'R1': 1, 'MK7': 1, 'MK6': 1, 'D1': 1, 'D2': 1, 'D3': 1, 'D4': 1, 'R6': 1, 'R7': 1, 'MK1': 1, 'MK2': 1, 'MK3': 1, 'MK4': 1, 'MK5': 1})
kitspace_dynamixel_shield
defaultdict(<class 'int'>, {'SHIELD1': 1, 'P4': 1, 'D1': 1, 'JP1': 1, 'JP2': 1, 'JP3': 1, 'JP4': 1, 'P2': 1, 'P3': 1, 'Q1': 1, 'R1': 1, 'R2': 1, 'U1': 1, 'U2': 1, 'P1': 1, 'F1': 1, 'R3': 1, 'P5': 1, 'P7': 1, 'P6': 1, 'P9': 1, 'P8': 1, 'R4': 1, 'R5': 1, 'R7': 1, 'R6': 1, 'C1': 1, 'C2': 1})
kitspace_esp8266
defaultdict(<class 'int'>, {'P1': 1, 'REF**': 11, 'C1': 1, 'D1': 1, 'J2':

('expects list of more than 2 element', [1437, 'layers'])
('expects list of more than 2 element', [1451, 'layers'])


linklayer_contact
defaultdict(<class 'int'>, {'P3': 1, 'JP1': 1, 'P1': 1, 'P2': 1, 'R1': 1, 'TP1': 1, 'TP2': 1, 'TP3': 1, 'U1': 1, 'U2': 1, 'U3': 1, 'X1': 1, 'C1': 1, 'C2': 1, 'C3': 1, 'C4': 1, 'C5': 1, 'C6': 1, 'C7': 1, 'C8': 1, 'C9': 1, 'C10': 1, 'C11': 1, 'C12': 1, 'C13': 1, 'C14': 1, 'C15': 1, 'C16': 1, 'D1': 1, 'D2': 1, 'R2': 1, 'R3': 1, 'R4': 1, 'SYM': 1, 'JP2': 1, 'JP3': 1, 'JP4': 1, 'JP5': 1, '""': 2})
oskirby_logicbone
defaultdict(<class 'int'>, {'IC3': 1, 'IC6': 1, 'J2': 1, 'J5': 1, 'R24': 1, 'J3': 1, 'D8': 1, 'IC7': 1, 'J4': 1, 'C169': 1, 'RN4': 1, 'C120': 1, 'J6': 1, 'C123': 1, 'IC5': 1, 'C122': 1, 'IC19': 1, 'C280': 1, 'IC14': 1, 'C542': 1, 'C150': 1, 'R3': 1, 'C435': 1, 'C469': 1, 'IC13': 1, 'P9': 1, 'P8': 1, 'C261': 1, 'C108': 1, 'C109': 1, 'C116': 1, 'C118': 1, 'C151': 1, 'C152': 1, 'C157': 1, 'C158': 1, 'C160': 1, 'C154': 1, 'C161': 1, 'C153': 1, 'C163': 1, 'C164': 1, 'C166': 1, 'C167': 1, 'C201': 1, 'C210': 1, 'C211': 1, 'C220': 1, 'C221': 1, 'C230': 1, 'C231': 1, 'C2

# Check FreeRouting results

In [2]:
import os

pcb_names = read_csv("route_fill_zones.csv")
results_folder = "./deleted_fill_zones/"
resuts_files = list(os.listdir(results_folder))
for pcb_name in pcb_names:
    res_name = pcb_name + ".ses"
    if res_name not in resuts_files:
        print(pcb_name)
    

kitspace_d20_r0.1
kitspace_d20_tri_r0.2
kitspace_d20_tri_r0.3
kitspace_EEZ%20DIB%20MCU%20r1B2
kitspace_glasgow
kitspace_lpa96
kitspace_mppt-1210-hus
kitspace_OtterCastAmp
kitspace_tri_panel_4up_r0.1
kitspace_ulx3s
kitspace_ulx4m
kitspace_USB-C-Screen-Adapter
LibreSolar_MPPT_charger_20A


# Update PCBs folder

Put processed.kicad_pcb into folder

In [4]:
import os
import shutil

good_raw = read_csv("./good_raw.csv")
fill_zones_pcbs = read_csv("./route_fill_zones.csv")
print(good_raw)
print(fill_zones_pcbs)

pcb_folder = '../../PCBs/'
for pcb in os.listdir(pcb_folder):
    target = '../../PCBs/' + pcb + '/processed.kicad_pcb'
    if pcb in good_raw:
        source = './good_raws/' + pcb + '.kicad_pcb'
        shutil.copy(source, target)
    elif pcb in fill_zones_pcbs:
        source = './deleted_fill_zones/' + pcb + ".kicad_pcb"
        shutil.copy(source, target)
    elif os.path.exists(target):
        os.remove(target)

    if os.path.exists('../../PCBs/' + pcb + '/error.log'):
        os.remove('../../PCBs/' + pcb + '/error.log')

    if os.path.exists('../../PCBs/' + pcb + '/final.json'):
        os.remove('../../PCBs/' + pcb + '/final.json')


['kitspace_antenna_analyser', 'bobc_control_panel', 'bobc_matrix_clock', 'bobc_LCD-panel-adapter-lvc', 'bobc_mbeduinopresso', 'bobc_MS-F100', 'cnlohr_wiflier', 'kitspace_bbq10breakout', 'kitspace_BQ25570_Harvester', 'kitspace_Brk40p', 'kitspace_CA3306E', 'kitspace_CH330', 'kitspace_ChaosLooper', 'kitspace_CO2', 'kitspace_CocoMixtape_UGM_Kicad', 'kitspace_cseduinov4', 'kitspace_DIY_detector', 'kitspace_dynamixel_shield', 'kitspace_esp8266', 'kitspace_esp8266-12-breakout', 'kitspace_ESP8266-PowerMonitor', 'kitspace_f-91w', 'kitspace_fast_diode_pcb', 'kitspace_filaSens', 'kitspace_gas_sensor', 'kitspace_GPSMux', 'kitspace_grove_adaptor', 'kitspace_hbridge_driver', 'kitspace_Heltec_Node', 'kitspace_hum_temp_sensor', 'kitspace_ir_sensor', 'kitspace_Mixtape_Barebone', 'kitspace_nunchuk_breakout', 'kitspace_OutlineChaos', 'kitspace_PCBs', 'kitspace_pmt_combiner', 'kitspace_postcard', 'kitspace_Potentiometer_mount_4LED', 'kitspace_Potentiometer_mount_8LED', 'kitspace_Potentiometer_mount_16LED'